In [1]:
import warnings;warnings.filterwarnings('ignore')

import re
import os
import copy
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import matplotlib.font_manager as fm
fe = fm.FontEntry(
    fname=r'/usr/share/fonts/truetype/nanum/NanumGothic.ttf', # ttf 파일이 저장되어 있는 경로
    name='NanumBarunGothic')                        # 이 폰트의 원하는 이름 설정
fm.fontManager.ttflist.insert(0, fe)              # Matplotlib에 폰트 추가
plt.rcParams.update({'font.size': 10, 'font.family': 'NanumBarunGothic'}) # 폰트 설정
plt.rc('font', family='NanumBarunGothic')

from sklearn.preprocessing import LabelEncoder, MultiLabelBinarizer

# 1.데이터 로드

In [2]:
data_path = '../data'
train_path = f'{data_path}/new/new_train_ver4.csv'
test_path  = f'{data_path}/new/new_test_ver4.csv'

train_data = pd.read_csv(train_path)
test_data = pd.read_csv(test_path)
print('Train data shape : ', train_data.shape, 'Test data shape : ', test_data.shape)

# train/test 구분을 위한 칼럼 생성.
train_data['is_test'] = 0
test_data['is_test'] = 1
df = pd.concat([train_data, test_data]) # 하나의 데이터로 합친다.

Train data shape :  (1118822, 35) Test data shape :  (9272, 34)


# 2.k-건설사

In [3]:
print(len(df['k-건설사'].unique()))
for c in list(df['k-건설사'].unique()):
    print(c)

193
우성건설
박연정
대한주택공사
현대건설
동양고속건설주식회사
대우건설
대림산업
삼성물산
unknown
현대,쌍용,GS
쌍용건설(주)
한보건설
삼성중공업
대성산업
중앙건설
세방기업
태영건설
신동아건설
고려개발(주)
대우건설,대림산업,태영건설
한라, 두진
고려산업개발(주)
롯데건설
GS건설
한화건설
한진중공업
(주)건영,삼익주택,라이프주택
계룡건설산업(주)
한양건설
건영
청담삼익아파트
계룡건설(주)
삼환
KR산업
동부건설
고속도로관리공단
SH공사
남광, 대우
청광종합건설(주)
중앙건설(주)
LG건설
신동아건설(주)
(주)청구주택
삼호건설
한솔건설
현대,대림
신동아 건설(주)
임광건설
벽산건설
현대 성우건설
홍익종합건업(주)
성지건설
남광토건(주)
신명건설,신원건설,미주건설
KCC건설
두산, 이수건설
두산건설
TEC건설
현대건설,현대산업개발,한진중공업
성원건설
라인건설
코오롱건설
금호
(주)대보건설
경남기업
서광산업
우림건설
청구건설
삼환, 삼익건설
(주)쌍용건설
보람건설(주)
한일건설
서희건설
대호아이피종합건설
라이프건설
갑을건설
SK건설
우성건설(주)
고려산업개발
인정건설
(주)우성건설
재건축조합
(주)월드건설
삼익건설
우리건설
대한전척(주)
포스코건설
풍림건설
(주)중앙건설
신도건설
풍림산업,현대건설
효성건설
현대엔지니어링
이수건설
한일개발
효성중공업
두산건설주식회사
세양건설
인덕주택재건축정비사업조합/(주)신도종합건설
롯데건설,우성건설
도시개발공사
대백건설
극동건설
BYC
신성건설
동일토건
(주) 신한건설
경동건설
남해건설
LIG건설
경향건설
라성건설
해태건설(주)
동양메이저건설
흑석5구역재개발조합
동원산업건설
쌍용건설
양우건설
대동건설
진흥기업
신창건설
유원, 선경, 대우건설
(주)우리창우
계룡건설
삼성물산, 대림산업
우정건설
고려개발
삼성물산, 현대산업개발
흥화건설
(주)대림/ (주)삼성
한보철강
아남건설주식회사
유원건설
진흥건설
우방건설
포스코,SK,현대
삼부토건
동아건설
돈암정릉구역주택재개발정비사업조합
동일하이빌
동선구역주택재개발조합
대명종합건설
대

In [4]:
# 동의어 사전
synonyms = {
    '삼성물산': ['삼성물산주식회사', '삼성물산(주)', '삼성물산 건설부문', '삼성물산(주) 건설부문', '삼성물산,두산건설', '삼성물산(주) 주택부문', "삼성건설", "삼성건설(주)", "삼성래미안", "삼성물산",
               "(주)삼성물산", "삼성", "삼성물산", "삼성중공업", "(주)삼성"],
    '현대건설': ['현대건설(주)', '현대건설주식회사', '(주)현대건설', '현대산업개발', '현대산업개발(주)', 'HDC현대산업개발', '현대bs&c', '현대BS&C', "(주)현대산업개발", "현대산업개발", 
               "주식회사 현대건설", "현대", "현대산업개발 주", "현대종합건설", "주시회사 현대건설", "현대산업개발", "현대엔지니어링", "현대힐스테이트"],
    '대림산업': ['대림산업(주)', '대림산업주식회사', '(주)대림산업', '대림건설', '대림건설(주)', "(주)대림", "대림", "대림산업", "대림산업 (주)", "대림산업"],
    '대우건설': ['대우건설(주)', '대우푸르지오', "(주)대우", "(주)대우건설", "대우건설", "주)대우건설", "대우경남", "대우", "대우자동차판매"],
    'GS건설': ['GS건설(주)', '지에스 건설', '지에스건설주식회사', '지에스건설(주)', "(주)gs공사", "GS 건설", "GS건설", "GS건설주식회사", "GS자이 건설", "gs건설", "럭키외", "GS"],
    '한라건설': ['(주)한라', '한라건설', '(주)한라건설', '한라'],
    '롯데건설': ['롯데건설(주)', '롯데건설주식회사', "(주)롯데건설", "롯데건설"],
    '한화건설': ['(주)한화건설', '한화건설(주)'],
    '두산건설': ['두산건설주식회사', '두산건설(주)', "두산", "두산건설", "두산건설(주)", "두산건설주식회사"],
    '우성건설': ['우성건설(주)', '(주)우성건설', '우성건설(주)'],
    '포스코건설': ['(주)포스코건설', '포스코, 포스코A&C', '포스코', '포스코'],
    'SH공사': ['sh공사', '에스에이치공사', 'SH', "SH공사", "sh 공사", "(주)에스에이치건설", "서울도시개발공사", "주택공사", "도시개발공사"],
    '건영': ['(주)건영', '(주) 건영', "(주) 건영", "건영", "건영건설", "건영종합건설(주)", " (주) 건영"],
    '신동아건설': ['신동아건설(주)', '신동아 건설(주)', '신동아', '신동아건설', '신동아건설(주)'],
    '계룡건설': ['계룡건설(주)', '계룡건설산업(주)'],
    '코오롱건설': ['코오롱글로벌(주)', '코오롱글로벌', '코오롱건설주식회사'],
    '성원건설': ['성원건설(주)', '(주)성원건설'],
    "금호건설": ["(주)금호", "(주)금호건설", "(주)금호산업", "금호", "금호건설", "금호건설(주)", "금호산업", "금호자이"],
    "동아건설": ["(주)동아건설산업", "동아건설", "동아건설산업(주)"],
    "동일토건": ["(주)동일토건", "동일토건"],
    "LG건설": ["엘지건설(주)"],
    "한일건설" : ["한일개발"],
    "삼호건설": ["삼호건설", "삼호주택", "(주)삼호"],
    "서광산업": ["(주)서광건설산업"],
    "서희건설": ["(주)서희건설", "서희건설", "주식회사 서희건설"],
    "신성건설": ["(주)신성건설", "신성건설", "신성건설(주)"],
    "신창건설": ["(주)신창건설", "신창건설", "신창건설(주)->우방건설"],
    "쌍용건설": ["(주)쌍용건설", "쌍용건설", "쌍용건설(주)", "쌍용건설(주)", "쌍용"],
    "우방건설": ["(주)우방", "우방", "우방건설"],
    "우리건설": ["(주)우리건설", "우리건설"],
    "일성건설": ["(주)일성건설", "일성건설", "(주)일성건설"],
    "임광건설": ["임광토건(주)"],
    "청구건설": ["청구", "청구주택"],
    "남광건설" : ["남광", "남광토건(주)", "남광토건"],
    "풍림건설" : ["풍림산업"],
    "중앙건설": ["(주)중앙건설", "중앙건설", "중앙건설(주)"],
    "진흥기업": ["(주)진흥기업", "진흥기업", "진흥기업(주)"],
    "태영건설": ["(주)태영", "(주)태영건설", "태영건설"],
    "한양건설": ["(주)한양", "(주)한양건설", "한양", "한양건설", "한양", "한양건설", "한양수자인", "한양주택 (주)", "?양건설", "한양산업개발", "한양산업개발(주"],
    "한진건설": ["(주)한진중공업", "한진중공업", "한진중공업(주)", "한신공영", "한신공영(주)", "한진중공업", "한진건설(주)"],
    "효성건설": ["(주)효성", "(주)효성건설", "효성건설", "효성중공업"],
    "KCC건설": ["KCC", "kcc건설"],
    "동부건설": ["동부건설", "동부건설(주)", "동부건설(주)외1"],
    "이수건설": ["이수건설", "이수건설(주)", "이수건설,대우엔지니어링"],
    "삼성중공업": ["삼성중공업", "삼성중공업(주)"],
    "호반건설" : ["(주)호반건설"],
    "LIG건설" : ["LIG건설", "LIG건설(주)"],
    "SK건설": ["(주)에스케이건설", "SK"],
    "경남기업": ["경남기업", "경남기업(주)"],
    "경향건설": ["경향건설(주)"],
    "계룡건설": ["계룡건설산업", "계룡건설(주)", "계룡건설산업(주)"],
    "고려개발": ["고려개발", "고려개발(주)", "고려산업개발", "고려산업개발(주)", "고려개발(주)", "고려산업개발", "고려산업개발(주)"],
    "극동건설" : ["극동건설", "극동건설(주)"],
    "나산건설" : ["나산종합건설"],
    "라이프건설": ["라이프주택"],
    "신도건설": ["(주)신도종합건설"],
    "쌍용건설": ["쌍용"],
    "우림건설": ["우림건설", "우림건설(주)"],
    "유원건설": ["유원건설", "유원건설(주)", '울트라건설', '울트라건설(주)', "유원울트라", "유원"],
    "월드건설": ['월드건설', '월드건설(주)'],
    '인정건설' : ['인정건설', '인정건설(주)'],
    "흥화건설": ["흥화공업", "흥화공업(주)"],
    "한솔건설": ["한솔"],
    "한보건설": ["한보", "한보철강", "한보철강"],
    "고려개발": ["고려개발 "],
    "동양메이저건설": ["동양메이저(주)", "동양고속건설주식회사"],
    "동일건설": ["동일토건", "동일하이빌", "동일", "동일하이빌"],
    "두산건설": ["두산", "두산중공업"],
    "미주실업": ["미주실업주식회사"],
    "벽산건설": ["벽산건설(주)"],
    "삼익건설": ["삼익주택(주)", "삼익주택", "삼익주택(주)", "삼익건설(주)"],
    "삼정건설": ["삼익건설(주)"],
    "삼익주택": ["삼익주택(주)"],
    "삼정건설": ["삼정기업"],
    "삼환건설": ["삼환기업", "삼환"],
    "세양건설": ["세양건설산업"],
    "신동아건설": ['신동아'],
    "양우건설": ['양우건설(주)', "양우건설사"],
    "우성건설": ["우성"],
    "우정건설": ["우정건설 주식회사"],
    "중앙건설": ["중앙하이츠", "중앙건설(주)"],
    "진흥건설": ["진흥기업", "진흥연립재건축조합", "진흥기업"],
    '풍림건설': ['풍림산업(주)', '풍림산업'],
    '한보건설': ['한보', '한보철상'],
    "신일건업": ['신일산업', '(주)신일건업'],
    "동일건설": ['동일토건', '동일'],
    '세양건설': ['세양건설산업(주)'],
    '라이프건설' : ['라이프주택'],
    '미주건설' : ['미주실업'],
    '선경건설' : ['선경'],
    "unknown" : ['(주)', '주식회사', '건설']
}

# (주) 제거하는 함수
def remove_corporate_suffix(name):
    return name.replace('(주)', '').strip()

# 여러 구분자를 처리하는 정규 표현식 패턴
def split_builders(builders):
    pattern = re.compile(r'[\/,\s]+')
    return pattern.split(builders.strip())

# 동의어 대체 함수
def replace_synonyms(value, synonyms_dict):
    for key, values in synonyms_dict.items():
        if value in values:
            return key
    return value

# 여러 문자열을 처리하는 함수
def process_builders(builders, synonyms_dict):
    builders_list = split_builders(builders)
    processed_list = [replace_synonyms(builder, synonyms_dict) for builder in builders_list]
    return processed_list

# (주) 제거
df['k-건설사'] = df['k-건설사'].apply(remove_corporate_suffix)

# 동의어 처리 및 리스트를 문자열로 변환
df['k-건설사'] = df['k-건설사'].apply(lambda x: process_builders(x, synonyms))

# MultiLabelBinarizer를 사용한 원-핫 인코딩
mlb = MultiLabelBinarizer()
df_encoded = pd.DataFrame(mlb.fit_transform(df['k-건설사']), columns=mlb.classes_, index=df.index)

# 인코딩된 데이터프레임 출력
print(df_encoded.columns)
print(df_encoded.head())

Index(['BYC', 'GS건설', 'JR건설', 'KCC건설', 'KR산업', 'LG건설', 'LIG건설', 'SH공사', 'SK건설',
       'TEC건설',
       ...
       '해태건설', '현대건설', '현산', '현진종합건설', '혜림건설', '호반건설', '홍익종합건업', '효성건설',
       '흑석5구역재개발조합', '흥화건설'],
      dtype='object', length=151)
   BYC  GS건설  JR건설  KCC건설  KR산업  LG건설  LIG건설  SH공사  SK건설  TEC건설  ...  해태건설  \
0    0     0     0      0     0     0      0     0     0      0  ...     0   
1    0     0     0      0     0     0      0     0     0      0  ...     0   
2    0     0     0      0     0     0      0     0     0      0  ...     0   
3    0     0     0      0     0     0      0     0     0      0  ...     0   
4    0     0     0      0     0     0      0     0     0      0  ...     0   

   현대건설  현산  현진종합건설  혜림건설  호반건설  홍익종합건업  효성건설  흑석5구역재개발조합  흥화건설  
0     0   0       0     0     0       0     0           0     0  
1     0   0       0     0     0       0     0           0     0  
2     0   0       0     0     0       0     0           0     0  
3     0   0       0    

In [5]:
print(df.shape)

(1128094, 36)


In [6]:
df.columns

Index(['본번', '부번', '아파트명', '전용면적', '층', '건축년도', '도로명', '해제사유발생일', '등기신청일자',
       '거래유형', '부동산유형', '분양형태', 'k-관리방식', 'k-복도유형', 'k-난방방식', 'k-전체동수',
       'k-전체세대수', 'k-건설사', 'k-사용검사일-사용승인일', 'k-연면적', 'k-주거전용면적', 'k-관리비부과면적',
       'k-등록일자', 'k-수정일자', '건축면적', '주차대수', '단지승인일', '좌표X', '좌표Y', '단지신청일',
       'target', '구', '동', '계약날짜', '계약날짜_경과일', 'is_test'],
      dtype='object')

In [7]:
df['k-건설사'].value_counts()

k-건설사
[unknown]       877637
[현대건설]           26942
[삼성물산]           26070
[대우건설]           16849
[GS건설]           12402
                 ...  
[한진건설, 한양건설]        25
[호반건설]              22
[신영건설]               3
[자이에스앤디]             2
[혜림건설]               1
Name: count, Length: 170, dtype: int64

In [8]:
# MultiLabelBinarizer를 사용한 원-핫 인코딩
mlb = MultiLabelBinarizer()
df_encoded = pd.DataFrame(mlb.fit_transform(df['k-건설사']), columns=mlb.classes_, index=df.index)

# 인코딩된 데이터프레임 출력
print(df_encoded.columns)
print(df_encoded.head())

Index(['BYC', 'GS건설', 'JR건설', 'KCC건설', 'KR산업', 'LG건설', 'LIG건설', 'SH공사', 'SK건설',
       'TEC건설',
       ...
       '해태건설', '현대건설', '현산', '현진종합건설', '혜림건설', '호반건설', '홍익종합건업', '효성건설',
       '흑석5구역재개발조합', '흥화건설'],
      dtype='object', length=151)
   BYC  GS건설  JR건설  KCC건설  KR산업  LG건설  LIG건설  SH공사  SK건설  TEC건설  ...  해태건설  \
0    0     0     0      0     0     0      0     0     0      0  ...     0   
1    0     0     0      0     0     0      0     0     0      0  ...     0   
2    0     0     0      0     0     0      0     0     0      0  ...     0   
3    0     0     0      0     0     0      0     0     0      0  ...     0   
4    0     0     0      0     0     0      0     0     0      0  ...     0   

   현대건설  현산  현진종합건설  혜림건설  호반건설  홍익종합건업  효성건설  흑석5구역재개발조합  흥화건설  
0     0   0       0     0     0       0     0           0     0  
1     0   0       0     0     0       0     0           0     0  
2     0   0       0     0     0       0     0           0     0  
3     0   0       0    

In [9]:
df = df.reset_index(drop=True)
df_encoded = df_encoded.reset_index(drop=True)

df = pd.concat([df, df_encoded], axis=1)
print(df.shape)

(1128094, 187)


In [10]:
train_df = df[df['is_test'] == 0]
test_df = df[df['is_test'] == 1]

train_df = train_df.drop(columns=['is_test'])
test_df = test_df.drop(columns=['is_test', 'target'])
print(train_df.shape, test_df.shape)

train_df.to_csv(f'{data_path}/new/new_train_ver5.csv', index=False)
test_df.to_csv(f'{data_path}/new/new_test_ver5.csv', index=False)

(1118822, 186) (9272, 185)


In [12]:
list(train_df.columns)

['본번',
 '부번',
 '아파트명',
 '전용면적',
 '층',
 '건축년도',
 '도로명',
 '해제사유발생일',
 '등기신청일자',
 '거래유형',
 '부동산유형',
 '분양형태',
 'k-관리방식',
 'k-복도유형',
 'k-난방방식',
 'k-전체동수',
 'k-전체세대수',
 'k-건설사',
 'k-사용검사일-사용승인일',
 'k-연면적',
 'k-주거전용면적',
 'k-관리비부과면적',
 'k-등록일자',
 'k-수정일자',
 '건축면적',
 '주차대수',
 '단지승인일',
 '좌표X',
 '좌표Y',
 '단지신청일',
 'target',
 '구',
 '동',
 '계약날짜',
 '계약날짜_경과일',
 'BYC',
 'GS건설',
 'JR건설',
 'KCC건설',
 'KR산업',
 'LG건설',
 'LIG건설',
 'SH공사',
 'SK건설',
 'TEC건설',
 'unknown',
 '갑을건설',
 '건설알포메',
 '건영',
 '경남기업',
 '경동건설',
 '경향건설',
 '계룡건설',
 '고려개발',
 '고려산업개발',
 '고속도로관리공단',
 '공영토건',
 '극동건설',
 '금하건설',
 '금호건설',
 '나산건설',
 '남광건설',
 '남해건설',
 '대동건설',
 '대림산업',
 '대명종합건설',
 '대백건설',
 '대보건설',
 '대성산업',
 '대우건설',
 '대주건설',
 '대창건설',
 '대한전척',
 '대한주택공사',
 '대호아이피종합건설',
 '도시공감',
 '돈암정릉구역주택재개발정비사업조합',
 '동방티앤씨',
 '동부건설',
 '동선구역주택재개발조합',
 '동성개발',
 '동아건설',
 '동양메이저건설',
 '동원산업건설',
 '동일건설',
 '동일토건',
 '동일하이빌',
 '두산건설',
 '두산건설주식회사',
 '두진',
 '라성건설',
 '라이프건설',
 '라인건설',
 '럭키개발',
 '롯데건설',
 '문영건설',
 '미원건설',
 '미주건설',
 '박연정',
 '벽산건설',
 '보람건설',
 '삼부토건',
 